<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-the-Data" data-toc-modified-id="Importing-the-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing the Data</a></span><ul class="toc-item"><li><span><a href="#Metadata-File" data-toc-modified-id="Metadata-File-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><code>Metadata</code> File</a></span></li><li><span><a href="#train_labels-File" data-toc-modified-id="train_labels-File-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span><code>train_labels</code> File</a></span></li><li><span><a href="#Prepping-the-data-for-the-Satellite-imagery-analysis." data-toc-modified-id="Prepping-the-data-for-the-Satellite-imagery-analysis.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Prepping the data for the Satellite imagery analysis.</a></span></li><li><span><a href="#Setting-up-the-DataFrame" data-toc-modified-id="Setting-up-the-DataFrame-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Setting up the DataFrame</a></span></li></ul></li><li><span><a href="#Temperature-Data" data-toc-modified-id="Temperature-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Temperature Data</a></span><ul class="toc-item"><li><span><a href="#different-approach-from-chatgpt" data-toc-modified-id="different-approach-from-chatgpt-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>different approach from chatgpt</a></span></li><li><span><a href="#This-is-now-from-chat-gpt" data-toc-modified-id="This-is-now-from-chat-gpt-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>This is now from chat gpt</a></span></li></ul></li></ul></div>

Running main notebook

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
import geopy.distance as distance

import planetary_computer as pc
from pystac_client import Client

from datetime import datetime, date
from datetime import timedelta

# from keras.utils import load_img, img_to_array
import requests
from PIL import Image
from io import BytesIO

from tqdm import tqdm
tqdm.pandas()

import rioxarray
import cv2
import odc.stac
import tempfile
import rasterio
import os

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import functions
import functions2

import warnings
warnings.filterwarnings('ignore')

# import cfgrib
# import boto3
# import netCDF4
import pygrib
import xarray as xr

# Importing the Data

## `Metadata` File

In [2]:
# Reading in the data and bringing in date as datetime dtype
metadata = pd.read_csv('Data/metadata.csv', parse_dates=['date'])

## `train_labels` File

In [3]:
train_labels = pd.read_csv('Data/train_labels.csv')

## Prepping the data for the Satellite imagery analysis.

In [4]:
sat_df = metadata

In [5]:
sat_df

,uid,latitude,longitude,date,split
0,aabm,39.080319,-86.430867,2018-05-14,train
1,aabn,36.559700,-121.510000,2016-08-31,test
2,aacd,35.875083,-78.878434,2020-11-19,train
3,aaee,35.487000,-79.062133,2016-08-24,train
4,aaff,38.049471,-99.827001,2019-07-23,train
...,...,...,...,...,...
23565,zzvv,36.708500,-121.749000,2014-12-02,test
23566,zzwo,39.792190,-99.971050,2017-06-19,train
23567,zzwq,35.794000,-79.012551,2015-03-24,train
23568,zzyb,35.742000,-79.238600,2016-11-21,train


In [6]:
sat_df['split'].value_counts()

train    17060
test      6510
Name: split, dtype: int64

In [7]:
sat_train = sat_df[sat_df['split'] == 'train'].copy()
sat_test = sat_df[sat_df['split'] == 'test'].copy()

Bringing back in the labels for sat_train.

In [8]:
sat_train = sat_train.merge(train_labels, on='uid')

## Setting up the DataFrame

Here I use a custom function to add a date range that the satellites can interpret and also include bounding boxes to later manipulate the images.

In [9]:
functions.get_important_info(sat_train, dist=31, big_crop_dist=3000, small_crop_dist=500, tiny_crop_dist=100);

In [10]:
sat_train.head(3)

,uid,latitude,longitude,date,split,region,severity,density,date_range,bbox,big_crop_bbox,small_crop_bbox,tiny_crop_bbox
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1,585.0,2018-04-29/2018-05-14,"[-87.00742888244132, 38.63091417147125, -85.85...","[-86.46553737052635, 39.05329612116674, -86.39...","[-86.43664511758249, 39.07581525298953, -86.42...","[-86.43202235685135, 39.079418305988106, -86.4..."
1,aacd,35.875083,-78.878434,2020-11-19,train,south,1,290.0,2020-11-04/2020-11-19,"[-79.43088170919651, 35.425434522510464, -78.3...","[-78.91165478658658, 35.84804560208817, -78.84...","[-78.88397103218583, 35.8705769758293, -78.872...","[-78.87954163128398, 35.87418198776951, -78.87..."
2,aaee,35.487000,-79.062133,2016-08-24,train,south,1,1614.0,2016-08-09/2016-08-24,"[-79.61191193921022, 35.03732231399556, -78.51...","[-79.09519299105324, 35.459960615407574, -79.0...","[-79.06764296370947, 35.48249344433146, -79.05...","[-79.06323495914324, 35.48609868913609, -79.06..."


In [11]:
functions.get_important_info(sat_test, dist=31, big_crop_dist=3000, small_crop_dist=500, tiny_crop_dist=100);

In [12]:
sat_test.head(3)

,uid,latitude,longitude,date,split,date_range,bbox,big_crop_bbox,small_crop_bbox,tiny_crop_bbox
1,aabn,36.559700,-121.51000,2016-08-31,test,2016-08-16/2016-08-31,"[-122.06728147535146, 36.11010270440991, -120....","[-121.5435112111439, 36.532665461053185, -121....","[-121.515585202077, 36.55519425204005, -121.50...","[-121.51111704041661, 36.558798850681036, -121..."
12,aair,33.042600,-117.07600,2014-11-01,test,2014-10-17/2014-11-01,"[-117.61009863726893, 32.59274323647839, -116....","[-117.10811706657985, 33.01554981570179, -117....","[-117.08135284459203, 33.038091644106125, -117...","[-117.07707056891928, 33.04169832908225, -117...."
14,aajw,40.703968,-80.29305,2015-08-26,test,2015-08-11/2015-08-26,"[-80.88337803767597, 40.25468908822623, -79.70...","[-80.32854854873526, 40.67695263815303, -80.25...","[-80.29896642510222, 40.69946544849961, -80.28...","[-80.29423328502216, 40.70306748998178, -80.29..."


# Temperature Data

In [13]:
sample_row = sat_train[sat_train['uid'] == 'garm']

In [14]:
sample_row

,uid,latitude,longitude,date,split,region,severity,density,date_range,bbox,big_crop_bbox,small_crop_bbox,tiny_crop_bbox
3950,garm,41.98006,-110.65734,2021-09-27,train,west,5,12267752.74,2021-09-12/2021-09-27,"[-111.25930520389633, 41.5308809188309, -110.0...","[-110.69353838433213, 41.953050646872555, -110...","[-110.6633730644045, 41.97555845000092, -110.6...","[-110.65854661288282, 41.97915969028358, -110...."


In [15]:
# cleaning sample row
sample_row = sample_row[['uid', 'date', 'latitude', 'longitude', 'tiny_crop_bbox']]
sample_row = sample_row.iloc[0]

In [16]:
sample_row

uid                                                            garm
date                                            2021-09-27 00:00:00
latitude                                                   41.98006
longitude                                                -110.65734
tiny_crop_bbox    [-110.65854661288282, 41.97915969028358, -110....
Name: 3950, dtype: object

In [17]:
# creating a sample date one day prior to real sample datetime
sample_date = sample_row['date'] - timedelta(1)

In [18]:
# converting to correct format
sample_date = f'{sample_date:%Y%m%d}'

In [19]:
sample_date

'20210926'

In [20]:
sample_bbox = sample_row['tiny_crop_bbox']

In [21]:
lat = sample_row.latitude
lon = sample_row.longitude

Using AWS to pull in data

Pulling in all climate data for specific sample date

In [23]:
sector = "conus"
cycle = 12  # noon
forecast_hour = 1  # offset from cycle time
product = "wrfsfcf"  # 3D Pressure Levels
sample_date = sample_date  # August 8 2019

file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
url = f"https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.{sample_date}/{sector}/{file_path}"
# print(f"Full URL: {url}")
print(str(url))

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20210926/conus/hrrr.t12z.wrfsfcf01.grib2


## different approach from chatgpt

In [23]:
# import s3fs

In [24]:
# rds = rioxarray.open_rasterio(url)

In [25]:
# import tempfile
# file = tempfile.NamedTemporaryFile(prefix="delete_later_", suffix='.grib2', delete=False)

# # headers = {"Range": f"bytes={range_start}-{range_end}"}
# resp = requests.get(url)

# with file as f:
#     f.write(resp.content)

## This is now from chat gpt

In [25]:
import tempfile
file = tempfile.NamedTemporaryFile(prefix="delete_later_", suffix='.grib2', delete=False)

# headers = {"Range": f"bytes={range_start}-{range_end}"}
resp = requests.get(url)

with file as f:
    f.write(resp.content)

In [26]:
grbs = pygrib.open(file.name)

In [34]:
# grbs[609].level

OSError: not that many messages in file

In [27]:
grbs.select()

[1:Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 2:3:3 (instant):lambert:cloudTop:level 0:fcst time 1 hrs:from 202109261200,
 3:201:201 (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 4:Vertically-integrated liquid:kg m**-1 (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 5:Visibility:m (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200,
 6:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 1000 m:fcst time 1 hrs:from 202109261200,
 7:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 4000 m:fcst time 1 hrs:from 202109261200,
 8:Derived radar reflectivity:dB (instant):lambert:isothermal:level 263 K:fcst time 1 hrs:from 202109261200,
 9:Wind speed (gust):m s**-1 (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200,
 10:U component of wind:m s**-1 (instant):lambert:isobaricInhPa:level 25000 Pa:f

In [28]:
surface_grb = grbs.select(level=0)

In [29]:
surface_grb

[1:Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 2:3:3 (instant):lambert:cloudTop:level 0:fcst time 1 hrs:from 202109261200,
 3:201:201 (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 4:Vertically-integrated liquid:kg m**-1 (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202109261200,
 5:Visibility:m (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200,
 9:Wind speed (gust):m s**-1 (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200,
 41:MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0:fcst time 1 hrs:from 202109261200,
 53:Hail:m (max):lambert:atmosphere:level 0 -:fcst time 0-1 hrs (max):from 202109261200,
 54:Hail:m (max):lambert:sigma:level 0.1 sigma value:fcst time 0-1 hrs (max):from 202109261200,
 55:Hail:m (max):lambert:surface:level 0:fcst time 0-1 hrs (max):from 202109261200,
 56:74:74 (max):lambert:atmosphereSingleLayer:lev

keep:
- 144
- 143
- 120
- 119
- 109 (weight of clouds)
- 107
- 88 (important, this is surface runoff)
- 84
- 83
- 64
- 62
- 9
- 5 (says visibility but could be indicator of particles in air)

In [30]:
temp_grb = grbs.select(name='Temperature')

In [31]:
temp_grb

[15:Temperature:K (instant):lambert:isobaricInhPa:level 50000 Pa:fcst time 1 hrs:from 202109261200,
 20:Temperature:K (instant):lambert:isobaricInhPa:level 70000 Pa:fcst time 1 hrs:from 202109261200,
 26:Temperature:K (instant):lambert:isobaricInhPa:level 85000 Pa:fcst time 1 hrs:from 202109261200,
 30:Temperature:K (instant):lambert:isobaricInhPa:level 92500 Pa:fcst time 1 hrs:from 202109261200,
 34:Temperature:K (instant):lambert:isobaricInhPa:level 100000 Pa:fcst time 1 hrs:from 202109261200,
 64:Temperature:K (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200]

In [32]:
temp_grb = temp_grb[-1]

In [33]:
temp_grb

64:Temperature:K (instant):lambert:surface:level 0:fcst time 1 hrs:from 202109261200

In [34]:
temp_grb.keys()

['globalDomain',
 'GRIBEditionNumber',
 'tablesVersionLatestOfficial',
 'tablesVersionLatest',
 'grib2divider',
 'angleSubdivisions',
 'missingValue',
 'ieeeFloats',
 'isHindcast',
 'section0Length',
 'identifier',
 'discipline',
 'editionNumber',
 'totalLength',
 'sectionNumber',
 'section1Length',
 'numberOfSection',
 'centre',
 'centreDescription',
 'subCentre',
 'tablesVersion',
 'masterDir',
 'localTablesVersion',
 'significanceOfReferenceTime',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'dataDate',
 'julianDay',
 'dataTime',
 'productionStatusOfProcessedData',
 'typeOfProcessedData',
 'md5Section1',
 'selectStepTemplateInterval',
 'selectStepTemplateInstant',
 'stepType',
 'is_chemical',
 'is_chemical_distfn',
 'is_chemical_srcsink',
 'is_aerosol',
 'is_aerosol_optical',
 'setCalendarId',
 'deleteCalendarId',
 'sectionNumber',
 'grib2LocalSectionPresent',
 'deleteLocalDefinition',
 'sectionNumber',
 'gridDescriptionSectionPresent',
 'section3Length',
 'numberOfSec

In [35]:
lats, lons = temp_grb.latlons()

In [36]:
lats.shape, lats.min(), lats.max(), lons.shape, lons.min(), lons.max()

((1059, 1799),
 21.138123000000004,
 52.61565330680793,
 (1059, 1799),
 -134.09547973426314,
 -60.91719277183784)

In [37]:
temp_grb.values.shape

(1059, 1799)

In [38]:
# lats = temp_grb.latitudes
# longs = temp_grb.longitudes
lats, longs = temp_grb.latlons()
temps = temp_grb.values

Setting up DFs

In [39]:
lats_df = pd.DataFrame(lats)

In [40]:
longs_df = pd.DataFrame(longs)

Setting up samples

In [41]:
sample_lat = sample_row.latitude

In [42]:
sample_long = sample_row.longitude

Lats_df

In [43]:
test_lats_df = lats_df[(lats_df >(sample_lat-.05)) & (lats_df < (sample_lat+.05))].dropna(how='all')

In [44]:
test_lats_df = test_lats_df.dropna(axis=1, how='all')

In [45]:
test_lats_df

,0,1,2,3,4,5,6,7,8,9,...,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798
657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,NaN,NaN,41.933616,41.943114,41.952602,41.962081,41.971551,41.981011,41.990462,41.999903,...,42.003175,41.993737,41.984289,41.974832,41.965366,41.955890,41.946405,41.936911,NaN,NaN
821,41.939783,41.949302,41.958813,41.968314,41.977805,41.987287,41.996760,42.006223,42.015677,42.025122,...,42.028395,42.018954,42.009503,42.000043,41.990573,41.981094,41.971606,41.962108,41.952601,41.943085
822,41.964970,41.974493,41.984007,41.993511,42.003006,42.012491,42.021967,NaN,NaN,NaN,...,NaN,NaN,NaN,42.025251,42.015778,42.006296,41.996805,41.987304,41.977793,41.968274
823,41.990156,41.999682,42.009199,42.018706,42.028204,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,42.022001,42.012497,42.002983,41.993460


Longs_df

In [46]:
test_longs_df = longs_df[(longs_df >(sample_long-.05)) & (longs_df < (sample_long+.05))].dropna(how='all')

In [47]:
test_longs_df = test_longs_df.dropna(axis=1, how='all')

In [48]:
test_longs_df

,437,438,439,440,441,442,443,444,445,446,...,583,584,585,586,587,588,589,590,591,592
0,-110.70588,-110.677696,-110.649510,-110.621320,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,-110.681761,-110.653566,-110.625368,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,-110.685828,-110.657624,-110.629418,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,-110.689897,-110.661685,-110.633471,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,-110.693969,-110.665749,-110.637526,-110.6093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-110.675420,-110.633382,NaN
1055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-110.681482,-110.639425,NaN
1056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-110.687549,-110.645473,NaN
1057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-110.693621,-110.651527,-110.609427


In [49]:
longs_matches = test_longs_df.where(test_longs_df.notnull()).where(test_lats_df.notnull()).dropna(how='all')

In [50]:
longs_matches.dropna(axis=1, how='all')

,536,537,538
683,-110.694255,-110.658418,-110.622577
684,-110.699429,-110.663579,-110.627724
685,-110.704608,-110.668744,-110.632876


In [51]:
sample_long

-110.65734

In [52]:
test_lats_df[[536, 537, 538]].dropna(how='all')

,536,537,538
683,41.946362,41.950204,41.954036
684,41.973015,41.976858,41.980691
685,41.999666,42.003511,42.007345
686,42.026316,NaN,NaN


In [53]:
temp_results = pd.DataFrame(temps)[[536,537, 538]].loc[[683, 684, 685]]

In [54]:
temp_results

,536,537,538
683,273.316284,273.441284,273.628784
684,272.816284,272.691284,273.441284
685,271.941284,272.066284,272.691284


In [55]:
temp_results.mean().mean()

272.8926730685764

Now to automate this:

In [56]:
climate_train_df = sat_train[['uid', 'date', 'latitude', 'longitude']]

In [57]:
climate_train_df['temp_1day'] = np.NaN

In [58]:
climate_train_df.head()

,uid,date,latitude,longitude,temp_1day
0,aabm,2018-05-14,39.080319,-86.430867,NaN
1,aacd,2020-11-19,35.875083,-78.878434,NaN
2,aaee,2016-08-24,35.487000,-79.062133,NaN
3,aaff,2019-07-23,38.049471,-99.827001,NaN
4,aafl,2021-08-23,39.474744,-86.898353,NaN


In [84]:
test_climate_train_df = climate_train_df.head(50)

In [85]:
test_climate_train_df

,uid,date,latitude,longitude,temp_1day
0,aabm,2018-05-14,39.080319,-86.430867,299.407155
1,aacd,2020-11-19,35.875083,-78.878434,287.311218
2,aaee,2016-08-24,35.487000,-79.062133,NaN
3,aaff,2019-07-23,38.049471,-99.827001,311.259369
4,aafl,2021-08-23,39.474744,-86.898353,312.196950
5,aafp,2017-11-15,35.647742,-79.271782,NaN
6,aagm,2020-06-10,35.906885,-79.132962,NaN
7,aahy,2014-08-12,35.726522,-79.125458,NaN
8,aaia,2018-06-27,35.980000,-78.791686,NaN
9,aaie,2013-11-06,35.861325,-78.768321,NaN


In [86]:
for index in range(len(test_climate_train_df)):
    row = test_climate_train_df.iloc[index]
    
    # getting temp data from one day prior
    sample_date = row['date'] - timedelta(1)
    sample_date = f'{sample_date:%Y%m%d}'
    
    # setting variables for lat/lon
    sample_lat = row.latitude
    sample_lon = row.longitude

    # setting up access point to NOAA HRRR
    sector = "conus" # continental united states
    cycle = 18  # noon CST (times are in UTC)
    forecast_hour = 1  # offset from cycle time
    product = "wrfsfcf"  # 2D Pressure Levels
    sample_date = sample_date  # sample date
    file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
    url = f"https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.{sample_date}/{sector}/{file_path}"
    
    # pulling the data from the request
    resp = requests.get(url)
    
    # setting up temp file it will delete when closed
    file = tempfile.NamedTemporaryFile(prefix="delete_later_", delete=False)
    
    # now pulling actual data
    with file as f:
        f.write(resp.content)
    try:
        grbs = pygrib.open(file.name)

        # grabbing surface temperature
        temp_grb = grbs.select(name='Temperature', level=0)[0]

        # getting lat/lon and temp matrices from results
        lats, lons = temp_grb.latlons()
        temps = temp_grb.values

        # setting up lat/lon dfs
        lats_df = pd.DataFrame(lats)
        lons_df = pd.DataFrame(lons)

        # slicing for location
        lats_df = lats_df[(lats_df >(sample_lat-.05)) & (lats_df < (sample_lat+.05))].dropna(how='all')
        lons_df = lons_df[(lons_df >(sample_lon-.05)) & (lons_df < (sample_lon+.05))].dropna(how='all')

        # getting matching coordinates
        matches = lons_df.where(lons_df.notnull()).where(lats_df.notnull()).dropna(how='all').dropna(axis=1, how='all')

        # getting temperature results
        temp_results = pd.DataFrame(temps)[matches.columns].loc[matches.index]

        # getting the average
        temp_results = temp_results.mean().mean()

        # adding results back to row
        test_climate_train_df['temp_1day'].loc[row.name] = temp_results
        grbs.close()
        file.close()
        os.remove(file.name)
    except:
        test_climate_train_df['temp_1day'].loc[row.name] = np.NaN
        grbs.close()
        file.close()
        os.remove(file.name)

In [87]:
test_climate_train_df

,uid,date,latitude,longitude,temp_1day
0,aabm,2018-05-14,39.080319,-86.430867,299.407155
1,aacd,2020-11-19,35.875083,-78.878434,287.311218
2,aaee,2016-08-24,35.487000,-79.062133,NaN
3,aaff,2019-07-23,38.049471,-99.827001,311.259369
4,aafl,2021-08-23,39.474744,-86.898353,312.196950
5,aafp,2017-11-15,35.647742,-79.271782,287.076609
6,aagm,2020-06-10,35.906885,-79.132962,307.212311
7,aahy,2014-08-12,35.726522,-79.125458,NaN
8,aaia,2018-06-27,35.980000,-78.791686,297.132619
9,aaie,2013-11-06,35.861325,-78.768321,NaN


In [77]:
row = test_climate_train_df.iloc[1]
# getting temp data from one day prior
sample_date = row['date'] - timedelta(1)
sample_date = f'{sample_date:%Y%m%d}'

# setting variables for lat/lon
sample_lat = row.latitude
sample_lon = row.longitude

# setting up access point to NOAA HRRR
sector = "conus" # continental united states
cycle = 18  # noon CST (times are in UTC)
forecast_hour = 1  # offset from cycle time
product = "wrfsfcf"  # 2D Pressure Levels
sample_date = sample_date  # sample date
file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
url = f"https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.{sample_date}/{sector}/{file_path}"

# pulling the data from the request
resp = requests.get(url)

# setting up temp file it will delete when closed
file = tempfile.NamedTemporaryFile(prefix="delete_later_", delete=False)

# now pulling actual data
with file as f:
    f.write(resp.content)
grbs = pygrib.open(file.name)



In [78]:
grbs.select()

[1:Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202011181800,
 2:3:3 (instant):lambert:cloudTop:level 0:fcst time 1 hrs:from 202011181800,
 3:Vertically-integrated liquid:kg m**-1 (instant):lambert:atmosphere:level 0 -:fcst time 1 hrs:from 202011181800,
 4:Visibility:m (instant):lambert:surface:level 0:fcst time 1 hrs:from 202011181800,
 5:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 1000 m:fcst time 1 hrs:from 202011181800,
 6:Derived radar reflectivity:dB (instant):lambert:heightAboveGround:level 4000 m:fcst time 1 hrs:from 202011181800,
 7:Derived radar reflectivity:dB (instant):lambert:isothermal:level 263 K:fcst time 1 hrs:from 202011181800,
 8:Wind speed (gust):m s**-1 (instant):lambert:surface:level 0:fcst time 1 hrs:from 202011181800,
 9:U component of wind:m s**-1 (instant):lambert:isobaricInhPa:level 25000 Pa:fcst time 1 hrs:from 202011181800,
 10:V component of wind:m s**-1 (instant):lambert:is

In [79]:
grbs.close()

In [80]:
file.close()

In [81]:
os.remove(file.name)